In [2]:
from LazyClassifierCustom import LazyClassifierCustom
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.inspection import DecisionBoundaryDisplay

from scipy import stats
from pca import pca
from IPython.display import display
import dataframe_image as dfi

from src.visualization import feature_importances_plot

import warnings
    
# warnings -> to silence warnings

warnings.filterwarnings("ignore")
np.set_printoptions(precision=5, suppress=True)


RANDOM_STATE = 42
N_JOBS = -1
class_names = ["Canis", "Dysg. Equisimilis", "Dysg. Dysgalactiae"]

map_target = {
    "Streptococcus canis": 0,
    "Streptococcus dysgalactiae subsp. equisimilis": 1,
    "Streptococcus dysgalactiae subsp. dysgalactiae": 2
}

map_target_inv = {
    0: "Strept. canis",
    1: "Strept. dysg. equisimilis",
    2: "Strept. dysg. dysgalactiae"
}

map_target_antibiotici = {
    "S" : 1,
    "NS" : 0
}
start = 9
n_antibiotici = 9
n_geni = 27
n_virulenza = 18
#n_picchi = ['46','306']
n_picchi = ['46']
N_BEST = 3

ModuleNotFoundError: No module named 'LazyClassifierCustom'

In [14]:
def makeDictBest(models, modelli, n_best = 5):
    for i in range(n_best):
        best = models.index[i]
        score = modelli.get(best)
        if score != None:
            modelli[best] = score + (n_best-i)
        else:
            modelli[best] = (n_best-i)
    return modelli

In [28]:
n_classes = [0,1]
# Hyperparameter tuning using RandomizedSearchCV
param_grid = {'LogisticRegression': {'C': np.logspace(-4, 4, 20), 
                                    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                                    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                                    'fit_intercept': [True, False],
                                    'intercept_scaling': [0.5, 1, 2],
                                    'class_weight': [None, 'balanced']},
              'RidgeClassifier' : {'alpha': np.logspace(-5, 5, 100)},
              'DecisionTreeClassifier': {'criterion': ['gini', 'entropy', 'log_loss'],
                                        'splitter': ['best', 'random'],
                                        'max_depth': [2*n for n in range(1,10)],
                                        'max_features': ['auto', 'sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'class_weight': [None, 'balanced']},
              'KNeighborsClassifier': {'n_neighbors': list(range(1, 20)),
                                        'weights': ['uniform', 'distance'],
                                        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                                        'p': [1,2]},
              'RandomForestClassifier': {'n_estimators': range(10, 100), 
                                        'max_features': ['auto', 'sqrt', 'log2'],
                                        'max_depth': [2*n for n in range(1,10)],
                                        'min_samples_split': range(2, 15), 
                                        'class_weight': [None, 'balanced'], 
                                        'criterion': ['gini', 'entropy', 'log_loss']},
              'BernoulliNB': {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                            'fit_prior': [True, False],
                            'class_prior': [None, [0.1,]* len(n_classes)],
                            'binarize': [None, -5, 0.0, 5, 10.0]
                            },
              'GaussianNB': {'var_smoothing': np.logspace(0,-9, num=20)},
              'NearestCentroid':  {'shrink_threshold': np.logspace(0, 1, 20),
                                   'metric': ['euclidean', 'manhattan']},
              'SVC': {'C': np.logspace(-3, 3, 10),
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                          'degree': range(2,5),
                          'gamma': np.logspace(-3, 1, 10)}
              }

In [30]:
for n in n_picchi:
    print('DATAFRAME CON '+n+' PICCHI')
    df = pd.read_csv("data/Dati_Matemaldomics_"+n+"picchi.csv",
                    delimiter=';', index_col='ID Strain')
    n = int(n)
    modelli = {}
    animal  = df[['Animal species of origin']]
    lancefield = df[['LANCEFIELD GROUP']]
    haemolysis = df[['Haemolysis']]
    subspecies = df[['Putative Subspecies']]

    st = df[[df.columns[4]]]
    maldi = df[df.columns[start:start+n]]
    antibiotici = df[df.columns[start+n:start+n+n_antibiotici]]
    geni_antibiotici = df[df.columns[start+n+n_antibiotici:start+n+n_antibiotici+n_geni]]
    virulenza = df[df.columns[start+n+n_antibiotici+n_geni:start+n+n_antibiotici+n_geni+n_virulenza]]
    
    maldi.fillna(0, inplace=True)
    maldi = maldi.replace(',', '.', regex=True)
    columns = maldi.columns
    for column in columns:
        maldi[column] = maldi[column].astype(float)
    display(maldi)
    
    targets = {#'antibiotici' : antibiotici,
                #'geni_antibiotici' : geni_antibiotici,
                'virulenza' : virulenza}
    
    feats_agg = {'lancefield' : lancefield,
                'haemolysis' : haemolysis,
                'subspecies' : subspecies,        
                'animal' : animal}
    
    for str_target,target in targets.items():
        columns = target.columns
        for column in columns:
            if str_target == 'antibiotici':
                target[column] = df[column].map(map_target_antibiotici)
            rapporto = (target[column] == 0).sum() / target.shape[0]
            #if (antibiotici[column] == 0).all() or (antibiotici[column] == 1).all():
            print(column+" : "+str(rapporto))
            if rapporto < 0.15 or rapporto > 0.85:
                target.drop([column], axis=1, inplace=True)
        
        display(target)
        
    metrics_df = pd.DataFrame(columns=['Target','Model','Accuracy','Precision','Recall','F1-Score','CV'])
    metrics_df_agg = pd.DataFrame(columns=['Target','Model','Accuracy','Precision','Recall','F1-Score','CV'])
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    targets['subspecies'] = subspecies
    targets['st'] = st
    X = maldi
    for str_target, target in targets.items():
        columns = target.columns
        for column in columns:    
            y = target[column]
            n_classes = np.unique(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
            clf = LazyClassifier(predictions=True)
            models, predictions = clf.fit(X_train, X_test, y_train, y_test)
            print("Colonna:"+column)
            display(models)
            print("\n")
            modelli = makeDictBest(models, modelli)
            for i in range(N_BEST):
                models_obj = clf.provide_models(X_train, X_test, y_train, y_test)
                model_name = models.index[i]
                print(model_name)
                model = models_obj[model_name]['classifier']
                print(model)
                params = param_grid.get(model_name)
                print(params)
                y_pred_lazy = predictions[model_name]
                print()
                acc = accuracy_score(y_test, y_pred_lazy)
                prec = precision_score(y_test, y_pred_lazy, average='weighted')
                rec = recall_score(y_test, y_pred_lazy, average='weighted')
                f1 = f1_score(y_test, y_pred_lazy, average='weighted')
                cv = cross_val_score(estimator=model, X=X_train, y=y_train,
                                                    scoring="accuracy", cv=skfold, n_jobs=N_JOBS, verbose=0).mean()
                ris = {'Target': column,
                        'Model': model_name,
                        'Accuracy' : acc,
                        'Precision' : prec,
                        'Recall' : rec,
                        'F1-Score' : f1,
                        'CV' : cv} 
                display(ris)
                metrics_df = metrics_df.append(ris, ignore_index=True)
                
                if params != None:
                    rs = RandomizedSearchCV(estimator=model, param_distributions=params,
                                        scoring="accuracy", n_jobs=-1, cv=skfold, verbose=1)
                    rs.fit(X_train, y_train)
                    parametri = rs.best_params_
                    best_model = rs.best_estimator_
                    cv_best = rs.best_score_
                else:
                    best_model = model
                    parametri = best_model.get_params()
                    cv_best = cross_val_score(estimator=model, X=X_train, y=y_train,
                                                    scoring="accuracy", cv=skfold, n_jobs=N_JOBS, verbose=0).mean()
                
                print(parametri)
                y_pred = best_model.predict(X_test)
                
                acc_best = accuracy_score(y_test, y_pred)
                prec_best = precision_score(y_test, y_pred, average='weighted')
                rec_best = recall_score(y_test, y_pred, average='weighted')
                f1_best = f1_score(y_test, y_pred, average='weighted')
                
                ris = {'Target': column,
                        'Model': model_name+'_Best',
                        'Accuracy' : acc_best,
                        'Precision' : prec_best,
                        'Recall' : rec_best,
                        'F1-Score' : f1_best,
                        'CV' : cv_best} 
                display(ris)
                metrics_df = metrics_df.append(ris, ignore_index=True)
            
            #models.to_csv('Risultati/LazyPredictor/model_'+str(n)+column+'.csv')
    del targets['subspecies']
    for str_feat, feat_agg in feats_agg.items():
        display(feat_agg)
        X = pd.concat([X, feat_agg], axis=1)
        for str_target, target in targets.items():
            columns = target.columns
            for column in columns:    
                y = target[column]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
                clf = LazyClassifier(predictions=True)
                models, predictions = clf.fit(X_train, X_test, y_train, y_test)
                print("Colonna: "+column+" con feat agg: "+str_feat)
                display(models)
                print("\n")
                modelli = makeDictBest(models, modelli)
                for i in range(N_BEST):
                    models_obj = clf.provide_models(X_train, X_test, y_train, y_test)
                    model_name = models.index[i]
                    print(model_name)
                    model = models_obj[model_name]['classifier']
                    preprocessing = models_obj[model_name]['preprocessing']
                    print(model)
                    params = param_grid.get(model_name)
                    print(params)
                    if params != None:
                        rs = RandomizedSearchCV(estimator=model, param_distributions=params,
                                            scoring="accuracy", n_jobs=-1, cv=skfold, verbose=1)
                        rs.fit(X_train, y_train)
                        parametri = rs.best_params_
                        print(parametri)
                        best_model = rs.best_estimator_
                        cv_best = rs.best_score_
                    else:
                        best_model = model
                        cv_best = cross_val_score(estimator=model, X=X_train, y=y_train,
                                                        scoring="accuracy", cv=skfold, n_jobs=N_JOBS, verbose=0).mean()
                    y_pred = best_model.predict(X_test)
                    
                    acc_best = accuracy_score(y_test, y_pred)
                    prec_best = precision_score(y_test, y_pred, average='weighted')
                    rec_best = recall_score(y_test, y_pred, average='weighted')
                    f1_best = f1_score(y_test, y_pred, average='weighted')
                    
                    ris = {'Target': column,
                            'Model': model_name,
                            'Accuracy' : acc_best,
                            'Precision' : prec_best,
                            'Recall' : rec_best,
                            'F1-Score' : f1_best,
                            'CV' : cv_best} 
                    display(ris)
                    metrics_df_agg = metrics_df_agg.append(ris, ignore_index=True)
                #models.to_csv('Risultati/LazyPredictor/model_'+str(n)+column+'_morefeat'+str_feat+'.csv')
    modelli = sorted(modelli.items(), key=lambda x:x[1])
    print(modelli)

DATAFRAME CON 46 PICCHI


,"2223,140967","2241,073989","2262,75751","2679,802856","2978,296408","3159,441237","3354,28405","3364,608472","3397,909861","3418,174965",...,"9030,351844","9073,208159","9487,183195","10103,20284","10400,80576","10491,16654","10930,54833","13276,73249","14943,03835","15048,89449"
ID Strain,,,,,,,,,,,,,,,,,,,,,
V13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V142,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V151,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V160,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V161,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V800,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


fbp54 : 0.0
gbs0630 : 0.9935064935064936
gbs0631 : 0.9935064935064936
gbs0632 : 0.9935064935064936
hasC : 0.0
lmb : 0.9935064935064936
mf2 : 0.961038961038961
mf3 : 0.6753246753246753
scpA : 0.9935064935064936
sda : 0.8766233766233766
ska : 0.9935064935064936
slo : 0.9935064935064936
smeZ : 0.9935064935064936
spec : 0.974025974025974
speg : 0.9090909090909091
spek : 0.961038961038961
spel : 0.974025974025974
spem : 0.948051948051948


,mf3
ID Strain,
V13,0
V142,1
V151,0
V160,0
V161,1
...,...
V800,0
V82,1
V90,0


100%|██████████| 29/29 [00:01<00:00, 27.59it/s]

...

100%|██████████| 29/29 [00:01<00:00, 15.23it/s]

Colonna: mf3 con feat agg: lancefield


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
SGDClassifier,0.84,0.85,0.85,0.84,0.02
NearestCentroid,0.81,0.83,0.83,0.81,0.03
LabelSpreading,0.84,0.82,0.82,0.84,0.04
RidgeClassifierCV,0.84,0.82,0.82,0.84,0.04
RidgeClassifier,0.84,0.82,0.82,0.84,0.02
PassiveAggressiveClassifier,0.84,0.82,0.82,0.84,0.03
LogisticRegression,0.84,0.82,0.82,0.84,0.04
LabelPropagation,0.84,0.82,0.82,0.84,0.02
XGBClassifier,0.84,0.79,0.79,0.84,0.11




SGDClassifier
SGDClassifier(random_state=42)
None


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "d:\PycharmProjects\Thesis-Streptococcus-Classification\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\PycharmProjects\Thesis-Streptococcus-Classification\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 890, in fit
    return self._fit(
  File "d:\PycharmProjects\Thesis-Streptococcus-Classification\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py", line 686, in _fit
    self._partial_fit(
  File "d:\PycharmProjects\Thesis-Streptococcus-Classification\venv\lib\site-packages\sklearn\line